In [ ]:
import asyncio
import nest_asyncio
from io import BytesIO

from playwright.async_api import async_playwright
from IPython.display import display
from PIL import Image

# Allow asyncio to work nicely inside Jupyter
nest_asyncio.apply()

URL = "https://steamdb.info/sales/?min_reviews=0"

async def test_playwright_page():
    global html  # ✅ 让这个函数给全局变量 html 赋值

    print("Launching Firefox in headless mode...")
    async with async_playwright() as p:
        browser = await p.firefox.launch(headless=True)
        page = await browser.new_page()
        
        print(f"Opening {URL} ...")
        await page.goto(URL, wait_until="networkidle")
        print("Page loaded (networkidle).")
        # wait until #DataTables_Table_0 loads
        await page.wait_for_selector("#DataTables_Tsdfsfsable_0")
        print("Table loaded. Taking screenshot...")
        

        # Take full-page screenshot (in memory only)
        screenshot_bytes = await page.screenshot(full_page=True)
        
        # Get HTML and text so we can inspect what really loaded
        html = await page.content()          # ✅ 把最终 HTML 存到全局变量 html
        body_text = await page.inner_text("body")

        await browser.close()
        print("Browser closed.\n")

    # Show screenshot in the notebook (本地调试好用，在 Actions 里无所谓)
    img = Image.open(BytesIO(screenshot_bytes))
    img.save("screenshot.png")
    display(img)

    # Show a snippet from the **middle** of the text, which is often more useful
    print("\n=== Middle text snippet from page body (for debugging) ===")
    if len(body_text) > 1200:
        mid = len(body_text) // 2
        snippet = body_text[mid - 600 : mid + 600]
    else:
        snippet = body_text
    print(snippet[:1000])  # cap to 1000 chars just in case

# Run the async function from Jupyter
loop = asyncio.get_event_loop()
loop.run_until_complete(test_playwright_page())



In [ ]:
from bs4 import BeautifulSoup
import pandas as pd
from datetime import date

# Parse the HTML we fetched earlier
soup = BeautifulSoup(html, "lxml")

# Each game row has <tr class="app" ...>
rows = soup.select("tr.app")
print(f"Found {len(rows)} rows with class 'app'.")

# Peek at the text of the first row so we know it looks right
if rows:
    first_row = rows[0]
    print("\n=== First row text preview ===")
    print(first_row.get_text(separator=" | ", strip=True)[:500])
else:
    print("No rows found — we may need to adjust the selector.")


In [ ]:
from datetime import date

today_str = date.today().isoformat()
records = []

print("Scraping game rows into a table...")

total_rows = len(rows)

for idx, row in enumerate(rows, start=1):
    # appid from the <tr> attribute
    appid = row.get("data-appid")

    # Steam store URL from the "info-icon" link
    store_link_tag = row.select_one("td a.info-icon")
    steam_store_url = store_link_tag["href"] if store_link_tag and store_link_tag.has_attr("href") else None

    # Game name from the bold link in the third <td>
    name_tag = row.select_one("td:nth-of-type(3) a.b") or row.select_one("a.b")
    name = name_tag.get_text(strip=True) if name_tag else None

    # Get all <td> cells so we can pull discount, price, rating by position
    cols = row.find_all("td")

    # --- Discount percent (numeric, e.g. 75) ---
    discount_percent = None
    if len(cols) > 3:
        disc_text = cols[3].get_text(strip=True)  # e.g. "-75%"
        if disc_text:
            disc_clean = disc_text.replace("%", "").replace("+", "").replace("-", "")
            try:
                discount_percent = float(disc_clean)
            except ValueError:
                discount_percent = None

    # --- Price (numeric if possible, else None) ---
    price = None
    if len(cols) > 4:
        price_text = cols[4].get_text(strip=True)  # e.g. "$9.99" or "Free"
        if price_text:
            # Remove common currency symbols and commas
            cleaned = price_text.replace("$", "").replace("€", "").replace("£", "").replace(",", "")
            try:
                price = float(cleaned)
            except ValueError:
                # For "Free", "Play For Free", etc., we'll keep it as None
                price = None

    # --- Rating percent (numeric) ---
    rating_percent = None
    if len(cols) > 5:
        # data-sort often has a clean numeric value (e.g. "70.58")
        rating_sort = cols[5].get("data-sort")
        if rating_sort:
            try:
                rating_percent = float(rating_sort)
            except ValueError:
                # Fallback to parsing the visible text like "70.58%"
                rating_text = cols[5].get_text(strip=True).replace("%", "")
                try:
                    rating_percent = float(rating_text)
                except ValueError:
                    rating_percent = None

    records.append({
        "date": today_str,
        "appid": appid,
        "name": name,
        "discount_percent": discount_percent,
        "rating_percent": rating_percent,
        "price": price,
        "steam_store_url": steam_store_url,
    })

    # Simple progress updates so you can see it's working
    if idx == 1 or idx == total_rows or idx % 20 == 0:
        print(f"  Processed {idx}/{total_rows} rows...")

# Turn into a DataFrame
df = pd.DataFrame(records)

print("\nDone! Preview of the first 5 rows:")
display(df.head())

print(f"\nDataFrame shape: {df.shape[0]} rows x {df.shape[1]} columns")


In [ ]:
csv_filename = "steamdb_sales.csv"

# Save to CSV (no index column)
df.to_csv(csv_filename, index=False)

print(f"Saved {df.shape[0]} rows to {csv_filename}")

# Quick preview to confirm it looks good
display(df.head())
